# Data analysis

**Goal**: Analyse the differences in hotel costs and per diems of employees of the commission

1. Import and explore the data we have scraped in the previous excercise.
2. Clean this data
3. Combine them with the [inflation index from Eurostat](https://ec.europa.eu/eurostat/databrowser/bookmark/f6a583fa-f744-4590-aa95-173aaa6ea3f1?lang=en) (grab the [direct link to the csv](https://ec.europa.eu/eurostat/api/dissemination/sdmx/3.0/data/dataflow/ESTAT/prc_hicp_midx/1.0/M.I05.CP11.*?c[geo]=BE,BG,CZ,DK,DE,EE,IE,EL,ES,FR,HR,IT,CY,LV,LT,LU,HU,MT,NL,AT,PL,PT,RO,SI,SK,FI,SE,IS,NO,CH,UK,ME,MK&compress=false&format=csvdata&formatVersion=2.0&c[TIME_PERIOD]=ge:2004-01+le:2023-10&lang=en&labels=name))
4. Analyse the data

##  1. Import and explore the scraped data

The first step for a data analysis is to have a close look at the data:
- What columns are there?
- How many rows?
- What is in the columns?

Pandas has helpful methods to do this.

```python
df.info() # information on the column names and non-zero values
df.sample(3) # returns 3 sample rows from the data
df.head(3) # returns the first 3 rows of the data
len(df) # returns the number of rows
df.columns # all the column names as a list
```

In [69]:
import pandas as pd

In [70]:
df = pd.read_csv("regulation_data.csv")
df.sample(3)

,Destination,Hotel ceiling,Daily allowance,Date
63,Greece,112,82,20220101
407,Luxembourg,145,92,20100101
578,Portugal,"68,91","124,89",20060101


In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Destination      636 non-null    object
 1   Hotel ceiling    636 non-null    object
 2   Daily allowance  636 non-null    object
 3   Date             636 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 20.0+ KB


### `df.info()` explained

![](img/info.png)

### Looking closely at the columns

To isolate columns using this syntax and look more closely at those we use the following syntax:

```python
df["column"]
```

This will return a `Series` from a `DataFrame`. It's important to know what data type you are working with, as all of them have their own methods.


```python
# for categorical variables
df["column"].unique() # all the unique values of the column
df["column"].value_counts() # how often does a value occur

# for numeric variables
df.hist(column='column') # makes a histogram
df.describe() # descriptive statistics for all numeric variables
df["column"].describe() # descriptive statistics for a single column
```

In [72]:
df["Destination"].value_counts()

Destination
Belgium            23
Latvia             23
United Kingdom     23
Sweden             23
Finland            23
Slovenia           23
Portugal           23
Poland             23
Austria            23
Netherlands        23
Malta              23
Hungary            23
Lithuania          23
Luxembourg         23
Cyprus             23
Germany            23
Italy              23
Czech Republic     23
France             23
Spain              23
Greece             23
Ireland            23
Denmark            23
Estonia            23
Romania            19
Bulgaria           19
Slovakia           14
Destination        11
Slovak Republic     9
Croatia             9
Destinations        3
Name: count, dtype: int64

This counts the values in the column. Note that we have the words *Destination* and *Destinations* in there. These are remnants of the headers, we don't want them. We also have *Slovakia* and *Slovak Republic* as a value. We need to make them one.

### Filtering the DataFrame

```python
df.query() # filters the DataFrame
```

Please see the documentation 

`.query()`

In [73]:
df2 = df.query("~Destination.str.contains('Destinations?')")

* `~` except
* `str.contains()` : pandas [str.contains()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.contains.html)
* ('Destinations **?**')

In [74]:
df2["Destination"] = df2["Destination"].replace({"Slovak Republic" : "Slovakia"})

/tmp/ipykernel_452118/4003084761.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Destination"] = df2["Destination"].replace({"Slovak Republic" : "Slovakia"})


In [75]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 622 entries, 0 to 635
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Destination      622 non-null    object
 1   Hotel ceiling    622 non-null    object
 2   Daily allowance  622 non-null    object
 3   Date             622 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 24.3+ KB


In [76]:
df2["Hotel ceiling"] = pd.to_numeric(df2["Hotel ceiling"].str.replace(",","."))
df2["Daily allowance"] = pd.to_numeric(df2["Daily allowance"].str.replace(",","."))

/tmp/ipykernel_452118/458300061.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Hotel ceiling"] = pd.to_numeric(df2["Hotel ceiling"].str.replace(",","."))
/tmp/ipykernel_452118/458300061.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Daily allowance"] = pd.to_numeric(df2["Daily allowance"].str.replace(",","."))


In [77]:
df2.describe()

,Hotel ceiling,Daily allowance,Date
count,622.000000,622.000000,6.220000e+02
mean,130.785225,92.640981,2.013750e+07
std,30.967412,23.799554,5.925305e+04
min,50.000000,52.000000,2.004050e+07
25%,115.000000,74.000000,2.009010e+07
50%,135.000000,92.000000,2.014050e+07
75%,150.000000,102.000000,2.019010e+07
max,209.000000,210.000000,2.023010e+07


In [78]:
df2.groupby("Destination").mean().sort_values("Hotel ceiling", ascending=False)

,Hotel ceiling,Daily allowance,Date
Destination,,,
United Kingdom,176.811739,116.656087,2.013553e+07
Sweden,161.814348,110.665652,2.013553e+07
Netherlands,156.468696,101.968696,2.013553e+07
Romania,153.894737,56.736842,2.015293e+07
Bulgaria,152.894737,57.526316,2.015293e+07
France,151.640870,98.035217,2.013553e+07
Denmark,151.395652,125.226522,2.013553e+07
Ireland,144.513913,110.172174,2.013553e+07
Luxembourg,137.956522,96.293913,2.013553e+07


In [79]:
df2.query("Destination == 'Sweden'")

,Destination,Hotel ceiling,Daily allowance,Date
26,Sweden,187.00,117.00,20230101
54,Sweden,187.00,117.00,20220701
82,Sweden,187.00,117.00,20220101
110,Sweden,187.00,117.00,20210101
138,Sweden,187.00,117.00,20200101
166,Sweden,187.00,117.00,20190101
194,Sweden,187.00,117.00,20180101
222,Sweden,187.00,117.00,20170101
250,Sweden,187.00,117.00,20160910
278,Sweden,160.00,97.00,20160101


In [81]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 622 entries, 0 to 635
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Destination      622 non-null    object 
 1   Hotel ceiling    622 non-null    float64
 2   Daily allowance  622 non-null    float64
 3   Date             622 non-null    int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 24.3+ KB


In [82]:
df2['Date'] = pd.to_datetime(df2['Date'], format="%Y%m%d")

/tmp/ipykernel_452118/2307183852.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Date'] = pd.to_datetime(df2['Date'], format="%Y%m%d")


In [84]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 622 entries, 0 to 635
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Destination      622 non-null    object        
 1   Hotel ceiling    622 non-null    float64       
 2   Daily allowance  622 non-null    float64       
 3   Date             622 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 24.3+ KB


In [90]:
df_inflation = pd.read_csv("inflation_data.csv")
df_inflation.sample(3).T

,5780,6081,814
STRUCTURE,dataflow,dataflow,dataflow
STRUCTURE_ID,ESTAT:PRC_HICP_MIDX(1.0),ESTAT:PRC_HICP_MIDX(1.0),ESTAT:PRC_HICP_MIDX(1.0)
STRUCTURE_NAME,HICP - monthly data (index),HICP - monthly data (index),HICP - monthly data (index)
freq,M,M,M
Time frequency,Monthly,Monthly,Monthly
unit,I05,I05,I05
Unit of measure,"Index, 2005=100","Index, 2005=100","Index, 2005=100"
coicop,CP11,CP11,CP11
Classification of individual consumption by purpose (COICOP),Restaurants and hotels,Restaurants and hotels,Restaurants and hotels
geo,NO,PL,CH


In [95]:
for column in df_inflation.columns:
    display(df_inflation[column].unique())
#df_inflation["Time frequency"].unique()

array(['dataflow'], dtype=object)

array(['ESTAT:PRC_HICP_MIDX(1.0)'], dtype=object)

array(['HICP - monthly data (index)'], dtype=object)

array(['M'], dtype=object)

array(['Monthly'], dtype=object)

...

array([nan])

array([ 96.99,  97.86,  97.64, ..., 151.8 , 150.8 , 146.7 ])

array([nan])

array([nan, 'u', 'd', 'du'], dtype=object)

array([nan, 'low reliability', 'definition differs (see metadata)',
       'definition differs (see metadata), low reliability'], dtype=object)